### NC Voting and Registration Data Analysis
- Data can be found here: https://www.ncsbe.gov/results-data
  - unique identifier = voter_reg_num + county_id
  - Registration Data Dictionary: https://s3.amazonaws.com/dl.ncsbe.gov/data/layout_ncvoter.txt

In [19]:
import pandas as pd
pd.set_option('float_format', '{:f}'.format)

In [4]:
history = pd.read_csv('data/ncvhis_Statewide.txt',sep='\t')
registration = pd.read_csv('data/ncvoter_Statewide.txt',sep='\t',encoding='unicode_escape',low_memory=False)

In [6]:
print(history.shape)
print(registration.shape)

(33867363, 15)
(8629236, 67)


In [51]:
registration.columns

Index(['county_id', 'county_desc', 'voter_reg_num', 'ncid', 'last_name',
       'first_name', 'middle_name', 'name_suffix_lbl', 'status_cd',
       'voter_status_desc', 'reason_cd', 'voter_status_reason_desc',
       'res_street_address', 'res_city_desc', 'state_cd', 'zip_code',
       'mail_addr1', 'mail_addr2', 'mail_addr3', 'mail_addr4', 'mail_city',
       'mail_state', 'mail_zipcode', 'full_phone_number', 'confidential_ind',
       'registr_dt', 'race_code', 'ethnic_code', 'party_cd', 'gender_code',
       'birth_year', 'age_at_year_end', 'birth_state', 'drivers_lic',
       'precinct_abbrv', 'precinct_desc', 'municipality_abbrv',
       'municipality_desc', 'ward_abbrv', 'ward_desc', 'cong_dist_abbrv',
       'super_court_abbrv', 'judic_dist_abbrv', 'nc_senate_abbrv',
       'nc_house_abbrv', 'county_commiss_abbrv', 'county_commiss_desc',
       'township_abbrv', 'township_desc', 'school_dist_abbrv',
       'school_dist_desc', 'fire_dist_abbrv', 'fire_dist_desc',
       'water_di

In [52]:
registration['year'] = pd.to_datetime(registration['registr_dt']).dt.year

In [155]:
registration[registration.year==2018]['status_cd'].value_counts(dropna=False)

A    348017
I    106024
R     21266
D      8204
S       996
Name: status_cd, dtype: int64

In [145]:
## Clean up blank values and change to undeclared
registration[['race_code','ethnic_code','gender_code']] = registration[['race_code','ethnic_code','gender_code']].replace(" ","U")
registration['race_code'] = 'race_' + registration['race_code']
registration['ethnic_code'] = 'ethnicity_' + registration['ethnic_code']
registration['gender_code'] = 'gender_' + registration['gender_code']
registration['party_cd'] = 'party_' + registration['party_cd']

Sum by County by Year
- party
- ethnicity
- race
- gender

In [149]:
aggregated = pd.DataFrame()
for col in ['party_cd','race_code','ethnic_code','gender_code']:
    temp = registration.groupby(['year','county_desc'])[col].value_counts(dropna=False).unstack(fill_value=0).reset_index()
    if col == 'party_cd':
        aggregated = temp
    else:
        aggregated = aggregated.merge(temp,how='left',on=['year','county_desc'])

In [150]:
aggregated

,year,county_desc,party_DEM,party_GRE,party_LIB,party_REP,party_UNA,race_A,race_B,race_I,...,race_O,race_P,race_U,race_W,ethnicity_HL,ethnicity_NL,ethnicity_UN,gender_F,gender_M,gender_U
0,1812,DURHAM,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
1,1812,IREDELL,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,1,0
2,1872,WASHINGTON,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,1,0,0
3,1900,ALAMANCE,86,0,0,62,21,0,10,0,...,0,0,2,157,0,158,11,108,60,1
4,1900,ALEXANDER,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6587,2022,WAYNE,668,0,27,829,1295,35,681,2,...,288,0,269,1539,255,1913,651,1272,1182,365
6588,2022,WILKES,149,0,11,698,578,10,56,2,...,63,0,111,1192,60,869,507,652,620,164
6589,2022,WILSON,614,0,14,484,942,12,636,4,...,162,0,301,930,153,946,955,852,841,361
6590,2022,YADKIN,109,0,4,447,393,4,31,0,...,78,0,92,747,83,433,437,434,406,113


In [163]:
aggregated.to_csv('nc_registration_data_by_year_by_county.csv')